In [7]:
# Standard Library Imports
import csv
import os
import pickle
import random
import sys
import time
import traceback
from datetime import date

# Third-Party Library Imports
import yaml
import pyautogui
import pdb  
from email_validator import validate_email, EmailNotValidError
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select, WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

# Local Module Imports
import config
import constants
import utils


In [1]:

# utils, constants, and config 모듈이 올바르게 정의되어 있어야 합니다.
# 예시로 간단히 정의합니다:
# import utils
# import constants
# import config

class Linkedin:
    def __init__(self):
        utils.prYellow("🤖 Thanks for using Easy Apply Jobs bot, for more information you can visit our site - www.automated-bots.com")
        utils.prYellow("🌐 Bot will run in Chrome browser and log in Linkedin for you.")
        self.driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=utils.chromeBrowserOptions())
        self.cookies_path = f"{os.path.join(os.getcwd(),'cookies')}/{self.getHash(config.email)}.pkl"
        self.driver.get('https://www.linkedin.com')
        self.loadCookies()

        if not self.isLoggedIn():
            self.driver.get("https://www.linkedin.com/login?trk=guest_homepage-basic_nav-header-signin")
            utils.prYellow("🔄 Trying to log in Linkedin...")
            try:
                self.driver.find_element(By.ID, "username").send_keys(config.email)
                time.sleep(2)
                self.driver.find_element(By.ID, "password").send_keys(config.password)
                time.sleep(2)
                self.driver.find_element(By.XPATH, '//button[@type="submit"]').click()
                time.sleep(30)
            except:
                utils.prRed("❌ Couldn't log in Linkedin by using Chrome. Please check your Linkedin credentials on config files line 7 and 8.")

            self.saveCookies()
        # start application
        self.linkJobApply()

    def getHash(self, string):
        return hashlib.md5(string.encode('utf-8')).hexdigest()

    def loadCookies(self):
        if os.path.exists(self.cookies_path):
            cookies = pickle.load(open(self.cookies_path, "rb"))
            self.driver.delete_all_cookies()
            for cookie in cookies:
                self.driver.add_cookie(cookie)

    def saveCookies(self):
        pickle.dump(self.driver.get_cookies(), open(self.cookies_path, "wb"))

    def isLoggedIn(self):
        self.driver.get('https://www.linkedin.com/feed')
        try:
            self.driver.find_element(By.XPATH, '//*[@id="ember14"]')
            return True
        except:
            pass
        return False

    def generateUrls(self):
        if not os.path.exists('data'):
            os.makedirs('data')
        try:
            with open('data/urlData.txt', 'w', encoding="utf-8") as file:
                linkedinJobLinks = utils.LinkedinUrlGenerate().generateUrlLinks()
                for url in linkedinJobLinks:
                    file.write(url + "\n")
            utils.prGreen("✅ Apply urls are created successfully, now the bot will visit those urls.")
        except:
            utils.prRed("❌ Couldn't generate urls, make sure you have editted config file line 25-39")

    def linkJobApply(self):
        self.generateUrls()
        countApplied = 0
        countJobs = 0

        urlData = utils.getUrlDataFile()

        for url in urlData:
            self.driver.get(url)
            time.sleep(random.uniform(1, constants.botSpeed))

            totalJobs = self.driver.find_element(By.XPATH, '//small').text
            totalPages = utils.jobsToPages(totalJobs)

            urlWords = utils.urlToKeywords(url)
            lineToWrite = "\n Category: " + urlWords[0] + ", Location: " + urlWords[1] + ", Applying " + str(totalJobs) + " jobs."
            self.displayWriteResults(lineToWrite)

            for page in range(totalPages):
                currentPageJobs = constants.jobsPerPage * page
                url = url + "&start=" + str(currentPageJobs)
                self.driver.get(url)
                time.sleep(random.uniform(1, constants.botSpeed))

                offersPerPage = self.driver.find_elements(By.XPATH, '//li[@data-occludable-job-id]')
                offerIds = [(offer.get_attribute("data-occludable-job-id").split(":")[-1]) for offer in offersPerPage]
                time.sleep(random.uniform(1, constants.botSpeed))

                for offer in offersPerPage:
                    if not self.element_exists(offer, By.XPATH, ".//*[contains(text(), 'Applied')]"):
                        offerId = offer.get_attribute("data-occludable-job-id")
                        offerIds.append(int(offerId.split(":")[-1]))

                for jobID in offerIds:
                    offerPage = 'https://www.linkedin.com/jobs/view/' + str(jobID)
                    self.driver.get(offerPage)
                    time.sleep(random.uniform(1, constants.botSpeed))

                    countJobs += 1

                    jobProperties = self.getJobProperties(countJobs)
                    if "blacklisted" in jobProperties:
                        lineToWrite = jobProperties + " | " + "* 🤬 Blacklisted Job, skipped!: " + str(offerPage)
                        self.displayWriteResults(lineToWrite)
                    else:
                        easyApplybutton = self.easyApplyButton()

                        if easyApplybutton is not False:
                            easyApplybutton.click()
                            time.sleep(random.uniform(1, constants.botSpeed))

                            try:
                                self.chooseResume()
                                questions = self.driver.find_elements(By.CLASS_NAME, 'fb-form-element')
                                handle_questions(questions)  # handle questions
                                self.driver.find_element(By.CSS_SELECTOR, "button[aria-label='Submit application']").click()
                                time.sleep(random.uniform(1, constants.botSpeed))

                                lineToWrite = jobProperties + " | " + "* 🥳 Just Applied to this job: " + str(offerPage)
                                self.displayWriteResults(lineToWrite)
                                countApplied += 1
                            except:
                                try:
                                    self.driver.find_element(By.CSS_SELECTOR, "button[aria-label='Continue to next step']").click()
                                    time.sleep(random.uniform(1, constants.botSpeed))
                                    self.chooseResume()
                                    questions = self.driver.find_elements(By.CLASS_NAME, 'fb-form-element')
                                    handle_questions(questions)  # handle questions
                                    comPercentage = self.driver.find_element(By.XPATH, 'html/body/div[3]/div/div/div[2]/div/div/span').text
                                    percenNumber = int(comPercentage[0:comPercentage.index("%")])
                                    result = self.applyProcess(percenNumber, offerPage)
                                    lineToWrite = jobProperties + " | " + result
                                    self.displayWriteResults(lineToWrite)
                                except Exception:
                                    self.chooseResume()
                                    lineToWrite = jobProperties + " | " + "* 🥵 Cannot apply to this Job! " + str(offerPage)
                                    self.displayWriteResults(lineToWrite)
                        else:
                            lineToWrite = jobProperties + " | " + "* 🥳 Already applied! Job: " + str(offerPage)
                            self.displayWriteResults(lineToWrite)

            utils.prYellow("Category: " + urlWords[0] + "," + urlWords[1] + " applied: " + str(countApplied) + " jobs out of " + str(countJobs) + ".")

        utils.donate(self)

    def chooseResume(self):
        try:
            self.driver.find_element(By.CLASS_NAME, "jobs-document-upload__title--is-required")
            resumes = self.driver.find_elements(By.XPATH, "//div[contains(@class, 'ui-attachment--pdf')]")
            if (len(resumes) == 1 and resumes[0].get_attribute("aria-label") == "Select this resume"):
                resumes[0].click()
            elif (len(resumes) > 1 and resumes[config.preferredCv-1].get_attribute("aria-label") == "Select this resume"):
                resumes[config.preferredCv-1].click()
            elif (type(len(resumes)) != int):
                utils.prRed("❌ No resume has been selected please add at least one resume to your Linkedin account.")
        except:
            pass

    def getJobProperties(self, count):
        textToWrite = ""
        jobTitle = ""
        jobLocation = ""

        try:
            jobTitle = self.driver.find_element(By.XPATH, "//h1[contains(@class, 'job-title')]").get_attribute("innerHTML").strip()
            res = [blItem for blItem in config.blackListTitles if (blItem.lower() in jobTitle.lower())]
            if (len(res) > 0):
                jobTitle += "(blacklisted title: " + ' '.join(res) + ")"
        except Exception as e:
            if (config.displayWarnings):
                utils.prYellow("⚠️ Warning in getting jobTitle: " + str(e)[0:50])
            jobTitle = ""

        try:
            time.sleep(5)
            jobDetail = self.driver.find_element(By.XPATH, "//div[contains(@class, 'job-details-jobs')]//div").text.replace("·", "|")
            res = [blItem for blItem in config.blacklistCompanies if (blItem.lower() in jobTitle.lower())]
            if (len(res) > 0):
                jobDetail += "(blacklisted company: " + ' '.join(res) + ")"
        except Exception as e:
            if (config.displayWarnings):
                print(e)
                utils.prYellow("⚠️ Warning in getting jobDetail: " + str(e)[0:100])
            jobDetail = ""
        except Exception as e:
            if (config.displayWarnings):
                print(e)
                utils.prYellow("⚠️ Warning in getting jobLocation: " + str(e)[0:100])
            jobLocation = ""

        textToWrite = str(count) + " | " + jobTitle +" | " + jobDetail + jobLocation
        return textToWrite

    def easyApplyButton(self):
        try:
            time.sleep(random.uniform(1, constants.botSpeed))
            button = self.driver.find_element(By.XPATH, "//div[contains(@class,'jobs-apply-button--top-card')]//button[contains(@class, 'jobs-apply-button')]")
            EasyApplyButton = button
        except: 
            EasyApplyButton = False

        return EasyApplyButton

    def applyProcess(self, percentage, offerPage):
        applyPages = math.floor(100 / percentage) - 2 
        result = ""
        for pages in range(applyPages):  
            self.driver.find_element(By.CSS_SELECTOR, "button[aria-label='Continue to next step']").click()

        self.driver.find_element( By.CSS_SELECTOR, "button[aria-label='Review your application']").click()
        time.sleep(random.uniform(1, constants.botSpeed))

        if config.followCompanies is False:
            try:
                self.driver.find_element(By.CSS_SELECTOR, "label[for='follow-company-checkbox']").click()
            except:
                pass

        self.driver.find_element(By.CSS_SELECTOR, "button[aria-label='Submit application']").click()
        time.sleep(random.uniform(1, constants.botSpeed))

        result = "* 🥳 Just Applied to this job: " + str(offerPage)

        return result

    def displayWriteResults(self,lineToWrite: str):
        try:
            print(lineToWrite)
            utils.writeResults(lineToWrite)
        except Exception as e:
            utils.prRed("❌ Error in DisplayWriteResults: " +str(e))

    def element_exists(self, parent, by, selector):
        return len(parent.find_elements(by, selector)) > 0

def handle_questions(questions):
    seen = set()
    
    for question in questions:
        if question in seen: continue
            
        try:
            question_label = question.find_element(By.CSS_SELECTOR, "label")
            input_field = question.find_element(By.CSS_SELECTOR, "input[type='text']")
        except:
            continue

        question_text = question_label.text.lower()
        is_required = input_field.get_attribute("required")
        
        if is_required == "false":
            continue  # Skip if the question is not required

        if 'how many year' in question_text:
            if not input_field.get_attribute('value'):
                input_field.send_keys('1')
                seen.add(question)
                      
        elif "city" in question_text:
            current_value = input_field.get_attribute("value")
            if not current_value:
                input_field.send_keys("Los Angeles")
                time.sleep(random.uniform(1, constants.botSpeed))
                input_field.send_keys(Keys.ARROW_DOWN, Keys.RETURN)
                seen.add(question)
                
    for question in questions:
        if question in seen: continue
        try:
            legend = question.find_element(By.CSS_SELECTOR, "legend span.fb-dash-form-element__label")
            radio_yes = question.find_element(By.CSS_SELECTOR, "input[data-test-text-selectable-option__input='Yes']")
            label_yes = question.find_element(By.CSS_SELECTOR, "label[data-test-text-selectable-option__label='Yes']")
        except:
            continue

        is_required = radio_yes.get_attribute("aria-required")
        if is_required == "false":
            continue

        if "degree" in legend.text.lower() or "master" in legend.text.lower() or "bachelor" in legend.text.lower():
            label_yes.click()
            seen.add(question)
    
    for question in questions:
        if question in seen: continue
        try:
            dropdown_label = question.find_element(By.CSS_SELECTOR, "label.fb-dash-form-element__label")
            dropdown = question.find_element(By.CSS_SELECTOR, "select[data-test-text-entity-list-form-select]")
        except:
            continue

        is_required = dropdown.get_attribute("aria-required")
        if is_required == "false":
            continue

        if "eligible to work" in dropdown_label.text.lower() or "job location" in dropdown_label.text.lower() or "responsible" in dropdown_label.text.lower() or "able" in dropdown_label.text.lower() or "willing" in dropdown_label.text.lower() or "comfortable" in dropdown_label.text.lower() or "open" in dropdown_label.text.lower() or "relocating" in dropdown_label.text.lower() or "passion" in dropdown_label.text.lower():
            select = Select(dropdown)
            select.select_by_visible_text("Yes")
            seen.add(question)
        elif "degree" in dropdown_label.text.lower():
            if "computer" in dropdown_label.text.lower():
                select = Select(dropdown)
                select.select_by_visible_text("Yes")
                seen.add(question)
            else:
                select = Select(dropdown)
                select.select_by_visible_text("No")
                seen.add(question)
                
    for question in questions:
        if question in seen: continue
        try:
            dropdown_label = question.find_element(By.CSS_SELECTOR, "label")
            dropdown = question.find_element(By.CSS_SELECTOR, "select")
        except:
            continue

        if "disability" in dropdown_label.text.lower():
            select = Select(dropdown)
            for option in select.options:
                if option.text.lower().startswith("no"):
                    select.select_by_visible_text(option.text)
                    seen.add(question)
                    break
                    
        elif "how" in dropdown_label.text.lower() and "hear" in dropdown_label.text.lower() and "about" in dropdown_label.text.lower():
            select = Select(dropdown)
            for option in select.options:
                if option.text.lower().startswith("linkedin"):
                    select.select_by_visible_text(option.text)
                    seen.add(question)
                    break
                    
        elif "veteran" in dropdown_label.text.lower():
            select = Select(dropdown)
            for option in select.options:
                if option.text.lower().startswith("i am not") or option.text.lower().startswith("no"):
                    select.select_by_visible_text(option.text)
                    seen.add(question)
                    break

    for question in questions:
        if question in seen: continue
        try:
            dropdown = question.find_element(By.CSS_SELECTOR, "select")
        except:
            continue

        select = Select(dropdown)
        
        for option in select.options:
            if 'asian' in option.text.lower():
                select.select_by_visible_text(option.text)
                seen.add(question)
                break
            elif option.text.lower().startswith("male"):
                select.select_by_visible_text(option.text)
                seen.add(question)
                break
            elif option.text.lower().startswith("man"):
                select.select_by_visible_text(option.text)
                seen.add(question)
                break    
            elif option.text.lower().startswith("he/him"):
                select.select_by_visible_text(option.text)
                seen.add(question)
                break
            elif option.text.lower().startswith("he, him"):
                select.select_by_visible_text(option.text)
                seen.add(question)
                break
            elif option.text.lower().startswith("i agree"):
                select.select_by_visible_text(option.text)
                seen.add(question)
                break    
            elif option.text.lower().startswith("i acknowledge"):
                select.select_by_visible_text(option.text)
                seen.add(question)
                break

    time.sleep(random.uniform(1, constants.botSpeed))

    # Additional questions handling logic
    for question in questions:
        if question in seen: continue
        
        # Locate the label for the "Yes" radio button inside the question
        label_yes = question.find_elements(By.XPATH, './/label[starts-with(@data-test-text-selectable-option__label, "I am not a protected veteran")]')
        if label_yes:
            label_yes[0].click()
            seen.add(question)
            continue

        label_yes = question.find_elements(By.XPATH, './/label[starts-with(@data-test-text-selectable-option__label, "No, I Don\'t Have A Disability, Or A History/Record Of Having A Disability")]')
        if label_yes:
            label_yes[0].click()
            seen.add(question)
            continue

        label_yes = question.find_elements(By.XPATH, './/label[starts-with(@data-test-text-selectable-option__label, "Male")]')
        if label_yes:
            label_yes[0].click()
            seen.add(question)
            continue

        label_yes = question.find_elements(By.XPATH, './/label[starts-with(@data-test-text-selectable-option__label, "Non-citizen allowed to work for any employer")]')
        if label_yes:
            label_yes[0].click()
            seen.add(question)
            continue

        label_yes = question.find_elements(By.XPATH, './/label[starts-with(@data-test-text-selectable-option__label, "Asian")]')
        if label_yes:
            label_yes[0].click()
            seen.add(question)
            continue

        label_yes = question.find_elements(By.XPATH, './/label[starts-with(@data-test-text-selectable-option__label, "East Asian")]')
        if label_yes:
            label_yes[0].click()
            seen.add(question)
            continue

        label_yes = question.find_elements(By.XPATH, './/label[starts-with(@data-test-text-selectable-option__label, "Not Hispanic or Latino")]')
        if label_yes:
            label_yes[0].click()
            seen.add(question)
            continue
            
    time.sleep(random.uniform(1, constants.botSpeed))


In [2]:
import time
import os
import pickle
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import hashlib
import math
from selenium.webdriver.common.keys import Keys
import utils
import constants


start = time.time()
linkedin_bot = Linkedin()  # Linkedin 인스턴스를 생성합니다.
linkedin_bot.linkJobApply()  # linkJobApply 메서드를 호출합니다.
end = time.time()
utils.prYellow("---Took: " + str(round((end - start)/60)) + " minute(s).")


🤖 Thanks for using Easy Apply Jobs bot, for more information you can visit our site - www.automated-bots.com
🌐 Bot will run in Chrome browser and log in Linkedin for you.


NameError: name 'config' is not defined